In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import csv
from datetime import date, datetime

In [2]:
options = webdriver.ChromeOptions()

# options.headless = True
options.add_argument('--ignore-certificate-errors-spki-list')
options.add_argument('--ignore-ssl-errors')

In [ ]:
flag = False
if os.path.exists('credentials.json'):
    try:
        with open('credentials.json', 'r') as f:
            values = json.loads(f.read())
            if len(values) == 0:
                flag = True
            elif 'Username' not in values.keys() or 'Password' not in values.keys():
                flag = True
    except json.JSONDecodeError:
        flag = True
else:
    flag = True

if flag:
    uname = input("Enter your haveloc username: ")
    passw = input("Enter your haveloc password: ")
    values = {}
    values['Username'] = uname
    values['Password'] = passw
    with open('credentials.json', 'w') as f:
        json.dump(values, f)

In [3]:
url = "https://app.haveloc.com/login"

driver = webdriver.Chrome("chromedriver", options=options)
driver.set_window_size(1920, 1080)

driver.get(url)

loginBox = driver.find_element("xpath", '//*[@id="root"]/div/div/form/div[1]/input')
loginBox.send_keys(values['Username'])

passBox = driver.find_element("xpath", '//*[@id="root"]/div/div/form/div[2]/input')
passBox.send_keys(values['Password'])

sleep(2)

source = driver.find_element("xpath",'//*[@id="root"]/div/div/form/div[3]/span')
action = webdriver.ActionChains(driver)
action.move_to_element(source).perform()

sleep(2)

xOffset = 0
yOffset = 100
webdriver.ActionChains(driver).move_by_offset(xOffset,yOffset).click().perform()

nextButton = driver.find_element("xpath", '//*[@id="root"]/div/div/form/button')
sleep(2)
nextButton.click()

# import pdb; pdb.set_trace()  
sleep(3)  

page_source = driver.page_source

soup = BeautifulSoup(page_source,features="html5lib")
PAGE_SIZE = int(''.join(filter(str.isdigit, soup.find_all("div", {"class": "bottomNavButtons"})[0].contents[1].contents[1].contents[0]))) + 1

fresherJobsurl = '''https://app.haveloc.com/FresherJobs?{"magicBoard":{
                                "pageSizeObj":'''+str(PAGE_SIZE)+''',
                                "filterObj":[],
                                "sortObj":[
                                    {
                                        "key":"jobApplyBy",
                                        "operation":"desc"
                                    }
                                ]
                            }
                        }
'''

driver.get(fresherJobsurl)

sleep(5)

page_source = driver.page_source

soup = BeautifulSoup(page_source,features="html5lib")

In [4]:
mydivs = soup.find_all("div", {"class": "mainCard"})
allCompanies = []
for i in mydivs:
    jobTitle = i.find("a",{"class": "cont1LeftTitle"}).contents[0]
    companyName = i.find("div", {"class": "cont1RightTitle"}).contents[0]
    companyLocation = i.find("div", {"class": "cont1LeftCollege"}).contents[0].strip()
    companyCTC = i.find_all("div", {"class": "singleTopicDesc"})[0].contents[0]

    applyBy = i.find_all("div", {"class": "singleTopicDesc"})[1].contents[0]
    month_map = {
        'jan': '01',
        'feb': '02',
        'mar': '03',
        'apr': '04',
        'may': '05',
        'jun': '06',
        'jul': '07',
        'aug': '08',
        'sep': '09',
        'oct': '10',
        'nov': '11',
        'dec': '12',
    }
    date, time = [j.strip() for j in applyBy.split('by')]
    date = date.split()
    date = date[0]+'-'+month_map[date[1].lower()]+'-'+date[2]

    jobStat = i.find_all("div", {"class": "jbStatTitle"})[0].contents
    jobCat = jobStat[0].contents[0].replace(',','').replace('#','').strip()
    jobStat = jobStat[1].strip()

    applicants = i.find_all("span", {"class": "hiredVsApp"})[0].contents[0].replace('Applicants','').strip()

    hrefs = i.find_all('a', href=True)
    jobApply = hrefs[0]['href']
    compDetails = hrefs[1]['href']
    
    allCompanies.append(
        {
            'Company Name': companyName,
            'Job Title': jobTitle,
            'CTC': companyCTC,
            'Apply By date': date,
            "Apply By time": time,
            "Company Location": companyLocation,
            "Job Category": jobCat,
            "Job status": jobStat,
            "Number of applicants": applicants,
            "jobApply": jobApply,
            "compDetails": compDetails
        }
    )

In [5]:
fieldnames = ['Company Name', 'Job Title', 'CTC', 'Apply By date', 'Apply By time', 'Company Location', 'Job Category', 'Job status', 'Number of applicants']

In [6]:
temp = []
for x in allCompanies:
    temp.append({key: x[key] for key in x if key not in ['jobApply', 'compDetails']})

with open('companies.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(temp)

In [20]:
baseUrl = 'https://app.haveloc.com'
appliable_companies = []

In [21]:
datetimern = datetime.now()

In [22]:
for i in temp:
    datetime_str = ''.join([x for x in i['Apply By date']])+":"+time.replace(":","-")
    datetime_object = datetime.strptime(datetime_str, '%d-%m-%Y:%H-%M') # create date and time object with the apply by time and date
    if datetime_object > datetimern:
        appliable_companies.append(i)
    else:
        break

In [23]:
appliable_companies

[{'Company Name': 'Byteridge',
  'Job Title': 'Byteridge - SDE 1',
  'CTC': '5.33 LPA',
  'Apply By date': '8-10-2022',
  'Apply By time': '15:00',
  'Company Location': 'full time ,  Hyderabad, Telangana, Bangalore, Karnataka, Delhi',
  'Job Category': 'Software Development',
  'Job status': 'IN PROGRESS',
  'Number of applicants': '147'},
 {'Company Name': 'dhiOmics',
  'Job Title': 'dhiOmics - Associate Machine Learning Scientist / Associate Data Engineer|| 23 M.Tech, B.Tech, M.Sc.',
  'CTC': '4.18 LPA',
  'Apply By date': '7-10-2022',
  'Apply By time': '18:00',
  'Company Location': 'full time , Hubli',
  'Job Category': '',
  'Job status': 'IN PROGRESS',
  'Number of applicants': '34'},
 {'Company Name': 'Tata Elxsi',
  'Job Title': 'Tata Elxsi - Engineer for UG & Senior Engineer for PG 2023 POB',
  'CTC': '5.5 LPA',
  'Apply By date': '7-10-2022',
  'Apply By time': '16:00',
  'Company Location': 'full time , Bangalore, Trivandrum, Pune, Chennai, Hyderabad & Kozhikode',
  'Job C

In [276]:
datetimern = datetime.now()

old = driver.window_handles
driver.execute_script("window.open('');")
new = driver.window_handles

combine = list(set(new) & set(old))
new_tab = list(set(new) - set(old))[0]

for tab in combine:
    driver.switch_to.window(tab)
    driver.close()

driver.switch_to.window(new_tab)

for i in allCompanies:
    datetime_str = ''.join([x for x in i['Apply By date']])+":"+time.replace(":","-")
    datetime_object = datetime.strptime(datetime_str, '%d-%m-%Y:%H-%M') # create date and time object with the apply by time and date
    if datetime_object > datetimern:
        print("\n=================================================================\n")
        print(f"Company Name: \t{i['Company Name']}")
        print(f"Job Title: \t{i['Job Title']}")
        print(f"CTC: \t\t{i['CTC']}")
        print(f"Deadline date: \t{i['Apply By date']}")
        print(f"Deadline time: \t{i['Apply By time']}\n")
        
        check = input("\nDo you want to apply to this company? (Y/N): ")
        
        if check.lower() == "y":
            
            driver.window_handles[0]
            driver.get(baseUrl+i['jobApply'])

            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])

            driver.get(baseUrl+i['compDetails'])
            input('Press Enter to continue to the next job application')
            
            old = driver.window_handles
            driver.execute_script("window.open('');")
            new = driver.window_handles
            
            combine = list(set(new) & set(old))
            new_tab = list(set(new) - set(old))[0]
            
            for tab in combine:
                driver.switch_to.window(tab)
                driver.close()
            
            driver.switch_to.window(new_tab)
            
            print("\n=================================================================\n")
            
            os.system('cls')



Company Name: 	Byteridge
Job Title: 	Byteridge - SDE 1
CTC: 		5.33 LPA
Deadline date: 	8-10-2022
Deadline time: 	15:00


Do you want to apply to this company? (Y/N): y
Press Enter to continue to the next job applicationn




Company Name: 	dhiOmics
Job Title: 	dhiOmics - Associate Machine Learning Scientist / Associate Data Engineer|| 23 M.Tech, B.Tech, M.Sc.
CTC: 		4.18 LPA
Deadline date: 	7-10-2022
Deadline time: 	18:00


Do you want to apply to this company? (Y/N): n


Company Name: 	Tata Elxsi
Job Title: 	Tata Elxsi - Engineer for UG & Senior Engineer for PG 2023 POB
CTC: 		5.5 LPA
Deadline date: 	7-10-2022
Deadline time: 	16:00


Do you want to apply to this company? (Y/N): n


In [26]:
import json, os

In [42]:
flag = False
if os.path.exists('credentials.json'):
    try:
        with open('credentials.json', 'r') as f:
            values = json.loads(f.read())
            if len(values) == 0:
                flag = True
            elif 'Username' or 'Password' not in values.keys():
                flag = True
    except json.JSONDecodeError:
        flag = True
else:
    flag = True

In [43]:
if flag:
    uname = input("Enter your haveloc username: ")
    passw = input("Enter your haveloc password: ")
    data = {}
    data['Username'] = uname
    data['Password'] = passw
    with open('credentials.json', 'w') as f:
        json.dump(data, f)

Enter your haveloc username: AM.EN.U4ELC19043
Enter your haveloc password: AM.EN.U4ELC1904317092001
